In [1]:
import numpy as np
import torch
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


In [2]:
def preprocess_data(csv_file, sequence_length):
    
  
    df = pd.read_csv(csv_file)
    
    features = df[['magnitude', 'cdi', 'mmi', 'dmin', 'gap', 'depth']].values
    
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(features)
    
    sequences = []
    for i in range(0, len(normalized_features) - sequence_length + 1, sequence_length // 2):  
        sequence = normalized_features[i:i + sequence_length]
        sequences.append(sequence)
    
    return torch.FloatTensor(sequences), scaler

In [3]:
normal_sequences, normal_scaler = preprocess_data("Normal.csv", 50)
anomalous_sequences, anomalous_scaler = preprocess_data("Anomalous.csv", 50)


/tmp/ipykernel_32427/2731436145.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400268359/work/torch/csrc/utils/tensor_new.cpp:261.)
  return torch.FloatTensor(sequences), scaler


In [4]:

from TsunoGan import TsunamiDetector


In [5]:
# Initialize the model
sequence_length = 50  # Sequence length for training (you already have it)
latent_dim = 100  # Latent dimension for the generator (can be adjusted)
lambda_ano = 0.1 # Weight for anomaly loss
num_features = 6 # Number of features (as in the input data)


# Create the TsunamiDetector instance
tsunami_detector = TsunamiDetector(sequence_length, latent_dim, lambda_ano, num_features,device='cpu')

# Train the model on the normal sequences (normal_sequences is already preprocessed)
batch_size = 64
epochs = 200

# Convert normal_sequences to a DataLoader compatible format (TensorDataset)
normal_data = TensorDataset(normal_sequences)
normal_dataloader = DataLoader(normal_data, batch_size=batch_size, shuffle=True)

# Train the model
tsunami_detector.train(normal_dataloader, epochs=epochs)



Epoch 0: G_loss: 0.5937, D_loss: 0.7089
Epoch 1: G_loss: 0.4684, D_loss: 0.7620
Epoch 2: G_loss: 0.4352, D_loss: 0.8343
Epoch 3: G_loss: 0.4104, D_loss: 0.8467
Epoch 4: G_loss: 0.3809, D_loss: 0.8784
Epoch 5: G_loss: 0.3798, D_loss: 0.9223
Epoch 6: G_loss: 0.3936, D_loss: 0.9075
Epoch 7: G_loss: 0.3718, D_loss: 0.8959
Epoch 8: G_loss: 0.3912, D_loss: 0.8905
Epoch 9: G_loss: 0.3967, D_loss: 0.8530
Epoch 10: G_loss: 0.4011, D_loss: 0.8274
Epoch 11: G_loss: 0.4444, D_loss: 0.8235
Epoch 12: G_loss: 0.4380, D_loss: 0.8198
Epoch 13: G_loss: 0.4490, D_loss: 0.7920
Epoch 14: G_loss: 0.5069, D_loss: 0.7601
Epoch 15: G_loss: 0.5248, D_loss: 0.7508
Epoch 16: G_loss: 0.5199, D_loss: 0.7211
Epoch 17: G_loss: 0.5455, D_loss: 0.7537
Epoch 18: G_loss: 0.5549, D_loss: 0.7016
Epoch 19: G_loss: 0.6166, D_loss: 0.6940
Epoch 20: G_loss: 0.6270, D_loss: 0.6923
Epoch 21: G_loss: 0.6830, D_loss: 0.6439
Epoch 22: G_loss: 0.6835, D_loss: 0.6480
Epoch 23: G_loss: 0.7059, D_loss: 0.6535
Epoch 24: G_loss: 0.6992, 

In [6]:
# Calibrate threshold with normal data
validation_data = normal_sequences  # You can use a validation set or a portion of normal data
threshold_percentile = 95
threshold = tsunami_detector.calibrate_threshold(validation_data, percentile=threshold_percentile)

print(f"Calibrated threshold: {threshold}")


Calibrated threshold: 3.5371963381767273


In [7]:
# Assuming anomalous_sequences is a tensor of anomalous data (like actual tsunami events)
anomalous_sequences = torch.FloatTensor(anomalous_sequences)  # Ensure it's a tensor if not already

# Evaluate anomalous data
for sequence in anomalous_sequences:
    result = tsunami_detector.detect_tsunami(sequence)
    print(f"ANOMALY: Is Tsunami: {result['is_tsunami']}, Confidence: {result['confidence']}, Anomaly Score: {result['anomaly_score']}")


ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.9925389289855957
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.3037679195404053
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.0036299228668213
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.732620120048523
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.857953667640686
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.9327208995819092
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.9574625492095947
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.6942704916000366
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.7511883974075317
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.6735271215438843
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.6825045347213745
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.6569664478302002


In [8]:
normal_sequences = torch.FloatTensor(normal_sequences)  # Ensure it's a tensor if not already

# Evaluate anomalous data
for sequence in normal_sequences:
    result = tsunami_detector.detect_tsunami(sequence)
    print(f"NORMAL: Is Tsunami: {result['is_tsunami']}, Confidence: {result['confidence']}, Anomaly Score: {result['anomaly_score']}")


NORMAL: Is Tsunami: True, Confidence: 1.0, Anomaly Score: 3.9682278633117676
NORMAL: Is Tsunami: True, Confidence: 1.0, Anomaly Score: 3.756676197052002
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.794037103652954
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.6136248111724854
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.1616199016571045
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.5907931327819824
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.5527909994125366
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.86222505569458
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.1870596408843994
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.7288196086883545
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.348418116569519
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.377158522605896
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.59431982040